# Load modules

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from keras.layers import Input, Dense
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from scipy.stats import percentileofscore

import pyod
from pyod.models.abod import ABOD
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.loci import LOCI
from pyod.models.lof import LOF
from pyod.models.lscp import LSCP
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.sos import SOS
from pyod.models.xgbod import XGBOD

import lime
import lime.lime_tabular
import shap
shap.initjs()
from helper.pdp import compute_pdp, plot_pdp, plot_ice

from time import time
from copy import deepcopy
import warnings
warnings.filterwarnings("default", category=FutureWarning, module='pyod')
warnings.filterwarnings("default", category=FutureWarning, module='sklearn')

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# Prepare input

In [ ]:
def stat_descr(arr, quantiles=[0, 0.25, 0.5, 0.75, 1.]):       
    n = len(arr)
    mu = np.mean(arr)
    std = np.std(arr)
    q_vals = np.quantile(arr, quantiles)
    
    def custom_format(x):
        if x > 50: return '{:^12.0f}'.format(x)
        else: return '{:^12.2f}'.format(x)
    q_str_perc = (''+'|').join(['{:8.0f}%   '.format(q*100) for q in quantiles])
    q_str_vals = (''+'|').join([custom_format(q) for q in q_vals])
    line = '-------------'*len(quantiles) 
    print(f'{n} values\n{mu:.3f} +/- {std:.3f}\n{q_str_perc}\n{line}\n{q_str_vals}\n')
    
# x = (np.random.randn(100)+5)
# stat_descr(x)
# stat_descr(x, quantiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])

In [ ]:
%%time 
df_orig = pd.read_csv('data/trending_merged_LHC18q_withGraphs.csv')

target_col = 'alias_global_Warning'
#----------

df = df_orig[[c for c in df_orig.columns if 
              ('gr' not in c and 'alias' not in c and 'Unnamed' not in c)
              and c != 'dataType.fString'
              or c == target_col
             ]]
rename = lambda c: c if c != target_col else 'bad'
df.columns = [rename(c) for c in df.columns]

good_idx = df['bad'] == 0
bad_idx  = df['bad'] == 1

In [ ]:
sens_vars = pd.read_csv('data/sensitive_variables_list.csv', sep=':')
sens_vars = [var[:-2] for var in sens_vars] + ['bad']

df = df[sens_vars]

# Train model

In [ ]:
%%time
# out_det = KNN(n_neighbors=3, method='mean')  # method, n_neighbours


data = df.drop(['bad'], axis=1)
data = data.drop(['oroc_A_side', 'oroc_C_side', 'iroc_A_side', 'iroc_C_side'], axis=1)
x = data.to_numpy()
scaler = StandardScaler(with_mean=False, with_std=False) 
x_s = scaler.fit_transform(x)
# data_s = (data - data.mean()) / data.std()
X = pd.DataFrame(x_s, columns=data.columns)
y = df['bad']
col_names = X.columns

# out_det = ABOD(n_neighbors=15)
# out_det = CBLOF()
# out_det = HBOS()
out_det = IForest(100, contamination=0.04)
# out_det = KNN(n_neighbors=3)
# out_det = LOCI()
# out_det = LOF()
# out_det = LSCP()
# out_det = MCD()
# out_det = OCSVM(gamma=0.1)
# out_det = PCA()
# out_det = SOS()
# out_det = XGBOD()

out_det.fit(X);
# scores = out_det.decision_scores_
scores = np.array([p[1] for p in out_det.predict_proba(X, method='linear')])
stat_descr(scores, quantiles=[0, 0.05, 0.25, 0.5, 0.75, 0.95, 1.])

bins = np.histogram(scores, 30)[1]
fig, axes = plt.subplots(2,2, figsize=(14,10))
# axes[0].hist(scores, bins=bins, histtype='step', lw=2, density=1, color='k');

axes[0][0].hist(scores, bins=bins, histtype='step', lw=2, density=1, color='k');
axes[0][0].set_title('normalized');

axes[0][1].hist(scores, bins=bins, histtype='step', lw=2, density=0, color='k');
axes[0][1].set_yscale("log", nonposy='clip')
axes[0][1].set_title('unnormalized (log y)');

axes[1][0].hist(scores[good_idx], bins=bins, histtype='step', lw=2, density=1, color='b');
axes[1][0].hist(scores[bad_idx], bins=bins, histtype='step', lw=2, density=1, color='r');
axes[1][0].set_title('normalized by class');

axes[1][1].hist(scores[good_idx], bins=bins, histtype='step', lw=2, density=0, color='b');
axes[1][1].hist(scores[bad_idx], bins=bins, histtype='step', lw=2, density=0, color='r');
axes[1][1].set_yscale("log", nonposy='clip')
axes[1][1].set_title('unnormalized (log y)');

In [ ]:
clf = out_det

# Explain model

## ~~Permutation importance~~

Permutation importance requires metric which is evaluated before and after permutation (drops in this metric correspond to feature importance).

One could think about permutation importance using raw scores returned by model,  
e.g. (`predict_proba` in `sklearn` or `decision_function` in `PyOD`) but it's not standard way of computing permutation importances.  
As single value is required to compute deviation from the baseline, one should use sth like:  
`aver( permutated_scores - original_scores )`  
The problem is that if features were independent then the importance of any feature will be equal to zero (contributions to various points' predictions will vanish in total) and it's very incorrect.  
Possible workaround would be to use `abs(permutated_scores - original_scores)`

In [ ]:
# def permutation_importances_custom(score_func, X):
#     baseline = score_func(X)
#     imp = []
#     imp_abs = []
#     for col in X.columns:
#         save = X[col].copy()
#         X[col] = np.random.permutation(X[col])
#         m = score_func(X)
#         X[col] = save
#         imp.append( np.mean(baseline - m) )
#         imp_abs.append( np.mean(abs(baseline - m)) )
#     return np.array(imp), np.array(imp_abs), X.columns.to_numpy()


# x1 = np.random.random_sample(1000)
# x2 = np.random.random_sample(1000)
# y = x1 + x2

# y = data_s['bad']
# X = data_s.drop('bad', axis=1)
# fimps, fimps_abs, fnames = permutation_importances_custom(score, X)
# idx = np.argsort(fimps_abs)
# idx.reverse()
# # idx = [i for i,_ in enumerate(fnames)]
# # for name, imp in zip(fnames, fimps): 
# for name, imp, imp_abs in zip(fnames[idx], fimps[idx], fimps_abs[idx]): 
#     print(f'{name:>25s}, {imp:>10.5f}, {imp_abs:>10.5f}')

## Compute SHAPley values

In [ ]:
# import shap

# # def score(X):
# #     return np.array([p[1] for p in clf.predict_proba(X)])

# col_names = X.columns
# # # kernel explainer with k-means
# X_summary_100 = shap.kmeans(X, 100)
# explainer_kmeans_100 = shap.KernelExplainer(clf.predict, X_summary_100)
# shap_values_kmeans_100 = explainer_kmeans_100.shap_values(X);  # ~2-4 it/sec for k=5-20 in k-means

In [ ]:
def score(X):
    return clf.predict_proba(X)[:,1]

# # kernel explainer with k-means
X_summary = shap.kmeans(X, 10)
explainer_kmeans = shap.KernelExplainer(score, X_summary)
shap_values_kmeans = explainer_kmeans.shap_values(X.iloc[:500]);  # ~2-4 it/sec for k=5-20 in k-means

In [ ]:
# %%time

# import pickle

# data = {'explainer_kmeans_100':explainer_kmeans_100, 'shap_values_kmeans_100':shap_values_kmeans_100,
#         'explainer_kmeans_10':explainer_kmeans_10, 'shap_values_kmeans_10':shap_values_kmeans_10}

# with open('shap_data.pickle', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# kernel explainer without k-means -- MAY BE VERY LOOOOOooooooooOOOOOOOOOOOOOOooooooOOOOOOOOooooooooooNG
# ~sec / it
# explainer_kernel = shap.KernelExplainer(clf.predict, X.iloc[:25])
# shap_values_kernel = explainer_kernel.shap_values(X.iloc[:25,])

## Most important variables from SHAP

In [ ]:
shap.summary_plot(shap_values_kmeans, 
                  X, 
                  feature_names=col_names,
                  plot_type='bar'
                 )
sorted_features_shap = col_names[abs(shap_values_kmeans).mean(axis=0).argsort()[::-1]].to_numpy()

## Advanced SHAP analyses

In [ ]:
shap.dependence_plot('zPullHighPt', 
                shap_values_kmeans, 
                X, 
                feature_names=col_names,
#                 interaction_index='fConstituents-0.fpT',
#                 interaction_index=None
                xmin=-6, xmax=5,
#                 alpha=0.3
                )

In [ ]:
shap.force_plot(explainer_kmeans.expected_value, 
                shap_values_kmeans, 
                X, 
                feature_names=col_names)

## Partial dependence plots

In [ ]:
# def partial_dependence_aver(clf, X, feat_name, percentiles_range=[0, 100], n_grid_points=50, scaler=None):
#     """computes partial dependence

#     Parameters
#     ----------
#     scaler : sklearn.preprocessing.StandardScaler or None
#         scaler obj to compute inverse transformation of `X` 
#         if None, then no transformation of X is computed

#     Returns
#     -------
#     grid : array, shape [n_grid_points]
#         values of `feat_name` for which partial dependece was computed
#         (x axis for pdp)
#     aver_score : array, shape [n_grid_points]
#         average score for samples with inputed values of `feat_name`
#         (y axis for pdp)
#     """
#     X_temp = X.copy()
    
#     grid = np.linspace(np.percentile(X_temp[feat_name], percentiles_range[0]),
#                        np.percentile(X_temp[feat_name], percentiles_range[1]),
#                        n_grid_points)
    
#     aver_pred = np.zeros(n_grid_points)
#     aver_score = np.zeros(n_grid_points)
    
#     orig_pred_aver  = np.average(clf.predict(X))
#     orig_pred_std   = np.std(clf.predict(X))
#     orig_score_aver = np.average(clf.decision_function(X))
#     orig_score_std  = np.std(clf.decision_function(X))
    
#     for i, val in enumerate(grid):
#         X_temp[feat_name] = val
#         aver_pred[i]  = (np.average(clf.predict(X_temp)) - orig_pred_aver) / orig_pred_std
#         aver_score[i] = (np.average(clf.decision_function(X_temp)) - orig_score_aver) / orig_score_std
        
#         if not i % 5: print(i, end=', ')
#     print()
            
#     if scaler:
#         feat_id = X.columns.tolist().index(feat_name)
#         grid_mat = np.zeros([n_grid_points, X.shape[1]])
#         grid_mat[:, feat_id] = grid
#         print(feat_id)
#         grid_mat_tr = scaler.inverse_transform(grid_mat)
#         grid = grid_mat_tr[:, feat_id]
            
# #         feat_mean = scaler.mean_[feat_id]
# #         feat_var  = scaler.var_[feat_id]
# #         grid      = grid * feat_var + feat_mean
        
#     return grid, aver_score

In [ ]:

ncols, nrows = 3,4
features = sorted_features_shap

def score_func(X,y):
    # does not use `y`
    return clf.predict_proba(X, method='linear')[:,1]

fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(20,15))
for r in range(nrows):
    for c in range(ncols):
        i = r*ncols + c
        feat_name = features[i]
        print(i)
        result = compute_pdp(score_func, 
                             X=X, 
                             feat_name=feat_name, 
                             quantiles_range=[0.005, 0.995],
                             n_grid_points=20, 
                             scaler=scaler)

        plot_pdp(result, 
                 plot_rugs=True, 
#                  plot_aver=True,
        #          instance_to_highlight=15,
                 centered=True, percentile_center_at=50, 
                 plot_lines=True, lines_frac_to_plot=100,
                 plot_errorband=False, errorband_nsigma=2,
                 ax=axes[r][c]
                )
        axes[r][c].set_title(feat_name)

In [ ]:
%%time

feature = 'meanMIP'

def score_func(X,y):
    # does not use `y`
    return clf.predict_proba(X, method='linear')[:,1]
    
result = compute_pdp(score_func, 
                     X=X, 
                     feat_name=feature, 
                     quantiles_range=[0.001, 0.999],
                     n_grid_points=20, 
                     scaler=scaler)

plot_pdp(result, 
         plot_rugs=True, 
         centered=False, percentile_center_at=50, 
         plot_lines=True, lines_frac_to_plot=25,
         plot_errorband=True, errorband_nsigma=2
        )

# Explain instance

## Instance overview

In [ ]:
np.argsort(clf.decision_scores_)[-200:]

In [ ]:
instance_index = 92

row_orig = df_orig.iloc[instance_index]
row = X.iloc[instance_index].to_numpy().reshape(1,-1)
global_warning_flag = df.iloc[instance_index]['bad']
model_pred = clf.predict(row)[0]
model_score = clf.decision_function(row)[0]
model_prob_linear  = clf.predict_proba(row, method='linear')[0][1]
model_prob_unify  = clf.predict_proba(row, method='unify')[0][1]

score_percentile = percentileofscore(clf.decision_scores_, model_score)

status_str =  f"chunk {instance_index} [ {row_orig['period.fString']} / {row_orig['run']} / chunk {row_orig['chunkID']} ]:  \n - _globalWarning_ flag set to: **{global_warning_flag:.0f}**  \n - model prediction is: **{model_pred}**  \n - model score: prob(lin) =  **{model_prob_linear:.3f}**, prob(unify) = {model_prob_unify:.3f} which is **{score_percentile:.2f}** percentile (higher = more outlier-like)"
printmd(status_str)

## LIME

In [ ]:
%%time

N_SAMPLES = 10000

n_top = 20
n_rep = 3

res = {}
print(f'\nsample id = {instance_index}')


for i in range(n_rep):
    print(i, end='...')
    explainer = lime.lime_tabular.LimeTabularExplainer(X.to_numpy(), feature_names=col_names, 
                                                       training_labels=df['bad'].to_numpy(),
                                                       class_names=['inlier', 'outlier'],
                                                       discretize_continuous=False, discretizer='entropy')

    exp = explainer.explain_instance(X.iloc[instance_index], 
                                     predict_fn=lambda X: clf.predict_proba(X, method='linear'), 
                                     num_features=50, num_samples=N_SAMPLES) # more features in order to fill values for less important feats also
#     exp.show_in_notebook(show_table=True, show_all=False)
#     exp.as_pyplot_figure()
#     for feat, val in exp.as_list():
#         if feat in res.keys(): res[feat].append(val)
#         else: res[feat] = [val,]
    for feat_id, val in exp.as_map()[1]:
        feat = col_names[feat_id]
        if feat in res.keys(): res[feat].append(val)
        else: res[feat] = [val,]

# PRINT RESULTS
print()
sorted_keys = sorted(res.keys(), key=lambda k: -abs(np.mean(res[k])))
for k in sorted_keys[:n_top]:
    v = res[k]
    print(f'{k:>50s} : {np.mean(v):2.4f} +/- {np.std(v):.4f} (n={len(v)})')

In [ ]:
top_feats = [(k, np.mean(res[k]), np.std(res[k])) for k in sorted_keys[:n_top]]
names = [xi[0] for xi in top_feats[:n_top]]
means = [xi[1] for xi in top_feats[:n_top]]
stds  = [xi[2] for xi in top_feats[:n_top]]
colors = ['r' if xi[1]>0 else 'b' for xi in top_feats[:n_top]]

plt.barh(range(n_top,0,-1), means, xerr=stds, color=colors)
locs, _ = plt.yticks()
plt.yticks(range(n_top,0,-1), names);

top_feats_lime = deepcopy(top_feats)

## SHAP

In [ ]:
n_top = 20
top_feats = sorted([el for el in zip(col_names, shap_values_kmeans[instance_index,])], key=lambda x: -abs(x[1]))
# print(x[:n_top])
names  = [xi[0] for xi in top_feats[:n_top]]
means  = [xi[1] for xi in top_feats[:n_top]]
colors = ['r' if xi[1]>0 else 'b' for xi in top_feats[:n_top]]

plt.barh(range(n_top,0,-1), means, color=colors)
locs, _ = plt.yticks()
plt.yticks(range(n_top,0,-1), names);

top_feats_shap = deepcopy(top_feats)

In [ ]:
# explainer_kmeans = explainer_kmeans_10
# shap_values_kmeans = shap_values_kmeans_10
shap.initjs()
shap.force_plot(explainer_kmeans.expected_value, 
                shap_values_kmeans[instance_index,:], 
                X.iloc[instance_index], 
                feature_names=col_names)

In [ ]:
%%time

top_feats = top_feats_shap

n_cols, n_rows = 3, 5
qmin, qmax = 0.005, 0.995
n_bins = 50
group_by_class = False

data_s = df

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16,4*n_rows))
# plt.subplots?
for c in range(n_cols):
    for r in range(n_rows):
        ax = axes[r][c]
        i = r*n_cols + c
        var_name = top_feats[i][0]
#         if 'roc' in var_name: continue
        minv = np.quantile(data_s[var_name], qmin)
        maxv = np.quantile(data_s[var_name], qmax)
        bins = np.linspace(minv, maxv, n_bins)
        if group_by_class:
            ax.hist(data_s.query('bad==0')[var_name], bins=bins, color='blue', histtype='step', density=1)
            ax.hist(data_s.query('bad==1')[var_name], bins=bins, color='red', histtype='step', density=1)
        else:
            ax.hist(data_s[var_name], bins=bins, color='green', histtype='step', density=1)
            
        ax.set_title(var_name)
        
        inst_val = data_s[var_name][instance_index]
        xrange = ax.get_xlim()[1] - ax.get_xlim()[0]
        yrange = ax.get_ylim()[1] - ax.get_ylim()[0]
        if inst_val > ax.get_xlim()[1] or inst_val < ax.get_xlim()[0]:
            mid = ax.get_xlim()[1]/2 + ax.get_xlim()[0]/2
            factor = 1 if inst_val > ax.get_xlim()[1] else -1
            dx = 0.2*xrange * factor
            arr_xpos = mid + 0.2*xrange*factor
            txt_xpos = arr_xpos - 0.1*xrange*factor
        else: 
            dx = 0
            arr_xpos = inst_val
            txt_xpos = arr_xpos + 0.1*xrange

        
        ax.arrow(arr_xpos, yrange, dx, -0.3*yrange, width=0.003*xrange, 
                length_includes_head=True, head_length=0.1*yrange, head_width=0.02*xrange,
                fc='k')
#         txt_xpos = inst_val+0.2 if (inst_val > minv and inst_val < maxv) else (maxv+minv)/1.5
#         txt_xpos = arr_xpos+0.2
        ax.text(txt_xpos, yrange*0.9, f'{inst_val:.3f}', fontsize=14, horizontalalignment='center')
    
        if top_feats[i][1] > 0: 
            ax.text(0.999, 1.2, '+', fontsize=30, color='red', transform=ax.transAxes, 
                    verticalalignment='top', horizontalalignment='right')
        elif top_feats[i][1] < 0:
            ax.text(0.98, 1.3, '-', fontsize=60, color='blue', transform=ax.transAxes, 
                    verticalalignment='top', horizontalalignment='right')
#         axes[r][c].legend?
    
plt.subplots_adjust(hspace=0.4)

## Marginal plots

In [ ]:
ncols, nrows = 3,5
top_feats = top_feats_shap


def score_func(X,y):
    # does not use `y`
    return clf.predict_proba(X, method='linear')[:,1]

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16,4*n_rows))
for r in range(nrows):
    for c in range(ncols):
        i = r*ncols + c
        ax = axes[r][c]
        feat_name = top_feats[i][0]
        print(i)
        result = compute_pdp(score_func, 
                             X=X, 
                             feat_name=feat_name, 
                             quantiles_range=[0.005, 0.995],
                             n_grid_points=20, 
                             scaler=scaler)

        plot_ice(result, instance_to_highlight=instance_index,
             plot_rugs=True, 
             plot_aver=False,
             centered=True, percentile_center_at=50, 
             plot_lines=False, lines_frac_to_plot=50,
             plot_errorband=False, errorband_nsigma=2,
             ax=ax
            )
        ax.set_title(feat_name)
        
        if top_feats[i][1] > 0: 
            ax.text(0.999, 1.1, '+', fontsize=30, color='red', transform=ax.transAxes, 
                    verticalalignment='top', horizontalalignment='right',
                   )
        elif top_feats[i][1] < 0:
            ax.text(0.98, 1.2, '-', fontsize=60, color='blue', transform=ax.transAxes, 
                    verticalalignment='top', horizontalalignment='right', 
                    )

In [ ]:
feature='meanTPCChi2'
result = compute_pdp(score_func, 
                     X=X, 
                     feat_name=feature, 
                     quantiles_range=[0.005, 0.995],
                     n_grid_points=50, 
                     scaler=scaler)

plot_ice(result, instance_to_highlight=instance_index,
         plot_rugs=True,
         plot_aver=True,
         centered=True, percentile_center_at=50, 
         plot_lines=True, lines_frac_to_plot=25,
         plot_errorband=False, errorband_nsigma=2
        )

## Display QA plots (per run)

In [ ]:
from IPython.display import Markdown as md

In [ ]:
available_plots = [('Event Information', 'TPC_event_info.png'), ('Cluster Occupancy', 'cluster_occupancy.png'), ('#eta, #phi and pt', 'eta_phi_pt.png'), ('Number of clusters in #eta and #phi', 'cluster_in_detail.png'), ('DCAs vs #eta', 'dca_in_detail.png'), ('TPC dEdx', 'TPC_dEdx_track_info.png'), ('DCAs vs #phi', 'dca_and_phi.png'), ('TPC-ITS matching', 'TPC-ITS.png'), ('dcar vs pT', 'dcar_pT.png'), ('Tracking parameter phi', 'pullPhiConstrain.png'), ('Raw QA Information', 'rawQAInformation.png'), ('Canvas ROC Status OCDB', 'canvasROCStatusOCDB.png'), ('Resolution vs pT and 1/pT', 'res_pT_1overpT.png'), ('Efficiency all charged + findable', 'eff_all+all_findable.png'), ('Efficiency #pi, K, p', 'eff_Pi_K_P.png'), ('Efficiency findable #pi, K, p', 'eff_Pi_K_P_findable.png')]
file_names_mapping = dict(available_plots)

row_orig = df_orig.iloc[instance_index]
path = '/'.join([str(el) for el in row_orig[['year', 'period.fString', 'pass.fString', 'run']].to_list()])
path = path.replace('pass1/', 'pass1/000')

def show_qa_plot(plot_name, path=path, file_names_mapping=file_names_mapping):
    plot_file_name = file_names_mapping[plot_name]
    src = f"http://aliqatpceos.web.cern.ch/aliqatpceos/data/{path}/{plot_file_name}"
    html = f'<img src={src} width="1200" height="1200">'
    print(src)
    return md(html)
    

interact(show_qa_plot, plot_name=file_names_mapping.keys(), 
         path=fixed(path), file_names_mapping=fixed(file_names_mapping));

In [ ]:
available_plots = [('Event Information', 'TPC_event_info.png'), ('Cluster Occupancy', 'cluster_occupancy.png'), ('#eta, #phi and pt', 'eta_phi_pt.png'), ('Number of clusters in #eta and #phi', 'cluster_in_detail.png'), ('DCAs vs #eta', 'dca_in_detail.png'), ('TPC dEdx', 'TPC_dEdx_track_info.png'), ('DCAs vs #phi', 'dca_and_phi.png'), ('TPC-ITS matching', 'TPC-ITS.png'), ('dcar vs pT', 'dcar_pT.png'), ('Tracking parameter phi', 'pullPhiConstrain.png'), ('Raw QA Information', 'rawQAInformation.png'), ('Canvas ROC Status OCDB', 'canvasROCStatusOCDB.png'), ('Resolution vs pT and 1/pT', 'res_pT_1overpT.png'), ('Efficiency all charged + findable', 'eff_all+all_findable.png'), ('Efficiency #pi, K, p', 'eff_Pi_K_P.png'), ('Efficiency findable #pi, K, p', 'eff_Pi_K_P_findable.png')]
file_names_mapping = dict(available_plots)

row_orig = df_orig.iloc[instance_index]
path = '/'.join([str(el) for el in row_orig[['year', 'period.fString', 'pass.fString', 'run']].to_list()])
path = path.replace('pass1/', 'pass1/000')

def show_qa_plot(plot_name, path=path, file_names_mapping=file_names_mapping):
    plot_file_name = file_names_mapping[plot_name]
    return md(f"![QA](http://aliqatpceos.web.cern.ch/aliqatpceos/data/{path}/{plot_file_name})")


# interact(show_qa_plot, plot_name=file_names_mapping.keys(), 
#          path=fixed(path), file_names_mapping=fixed(file_names_mapping));

In [ ]:
# h = """<div align="left"><br>
# <h2>Run Data Quality</h2>
# <a href="TPC_event_info.png">Event Information</a><br>
# <a href="cluster_occupancy.png">Cluster Occupancy</a><br>
# <a href="eta_phi_pt.png">#eta, #phi and pt</a><br>
# <a href="cluster_in_detail.png">Number of clusters in #eta and #phi</a><br>
# <a href="dca_in_detail.png">DCAs vs #eta</a><br>
# <a href="TPC_dEdx_track_info.png">TPC dEdx</a><br>
# <a href="dca_and_phi.png">DCAs vs #phi</a><br>
# <a href="TPC-ITS.png">TPC-ITS matching</a><br>
# <a href="dcar_pT.png">dcar vs pT</a><br>
# <a href="pullPhiConstrain.png">Tracking parameter phi</a><br>
# <a href="rawQAInformation.png">Raw QA Information</a><br>
# <a href="canvasROCStatusOCDB.png">Canvas ROC Status OCDB</a><br>
# <!--  <a href="res_pT_1overpT.png">Resolution vs pT and 1/pT</a><br>  //-->
# <!--  <a href="eff_all+all_findable.png">Efficiency all charged + findable</a><br>  //-->
# <!--  <a href="eff_Pi_K_P.png">Efficiency #pi, K, p</a><br>  //-->
# <!--  <a href="eff_Pi_K_P_findable.png">Efficiency findable #pi, K, p</a><br>  //-->
# </div>"""

# img_names = []
# for line in h.split('<br>'):
#     print(line)
#     if 'href' in line:
#         img_file_name = line.split('"')[1]
#         img_name = line.split('png">')[1].split('<')[0]
#         print('img_name = ',img_name, '>>', img_file_name)
#         img_names.append((img_name, img_file_name))

# TODO
- investigate usage of PyOD_clf.predict_proba(X, method='unify') -- everywhere (also in issues below)
- investigate LIME's parameters
- investigate SHAP coherence for various `k` in k-means and various explainers